*upd. 14-feb-2026*
Se seleccionaron tres estaciones para hacer una evaluación preliminar de los índices SPEI y SPI. Estas están distribuidas en el estado de Jalisco, específicamente, en la zona costa (Puerto Vallarta), centro (Guadalajara) y altos (Lagos Moreno). En esta primera parte del script, se hace la limpieza y análisis exploratorio de los CSV descargados del SMN.

In [3]:
#se importan las librerias
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

##### Limpieza de los datos
1. Se usa `skiprows`y `usecols`para leer desde el principio solo la información necesaria
2. Se convierten los valores con la etiqueta NULOS a *NaN*
3. Se elimina la fila que contiene strings con las unidades para que no estorben en futuros cálculos

In [4]:
def carga_smn(id_estac):
    ruta = f"../01_DATA/raw/dia{id_estac}.txt"

    #Leer los archivos evitando los metadatos para que no estorben al hacer lo cálculos
    df = pd.read_csv(ruta, sep='\\s+', na_values=['NULO'], usecols = ('FECHA', 'PRECIP','TMAX', 'TMIN'),skiprows=19)
    
    #Se elimina la fila de las unidades por el mismo motivo anterior
    df = df.drop(index=0).reset_index(drop=True)
        
    #poner columnas en el formato adecuado de variable
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    cols_df = ['PRECIP', 'TMAX', 'TMIN']
    for i in cols_df:
        df[i] = pd.to_numeric(df[i], errors = 'coerce')
    return df

# ---- Para agilizar. se usa un bucle
#estacs = ["14339","14067", "14024", #costa
#          "14066", "14065", "14169", #centro
#          "14084", "14083"] #altos

estacs = ["14002", "14006", "14009", "14011" , "14016" , "14017" , "14018" , "14023" , "14024",
    "14026" , "14028" , "14029" , "14030" , "14032" , "14033" , "14034" , "14035" , "14036" , "14037",
    "14038" , "14039" , "14040" , "14044" , "14046" , "14047" , "14048" , "14052" , "14053" , "14054",
    "14056" , "14059" , "14060" , "14063" , "14065" , "14066" , "14067" , "14068" , "14070" , "14071",
    "14072" , "14075" , "14076" , "14078" , "14080" , "14081" , "14083" , "14084" , "14087" , "14089",
    "14090" , "14093" , "14096" , "14099" , "14100" , "14101" , "14104" , "14111" , "14113" , "14114",
    "14117" , "14118" , "14119" , "14122" , "14123" , "14125" , "14129" , "14132" , "14136" , "14139",
    "14140" , "14141" , "14142" , "14143" , "14144" , "14145" , "14146" , "14155" , "14156" , "14157",
    "14160" , "14165" , "14167" , "14168" , "14169" , "14171" , "14179" , "14180" , "14187" , "14189",
    "14195" , "14197" , "14198" , "14199" , "14200" , "14202" , "14266" , "14269" , "14297" , "14306",
    "14311" , "14320" , "14323" , "14324" , "14326" , "14329" , "14331" , "14336" , "14337" , "14339",
    "14343" , "14346" , "14348" , "14349" , "14350" , "14351" , "14355" , "14367" , "14368" , "14369",
    "14379" , "14386" , "14391" , "14392" , "14396" , "14397"]

# se hace un diccionario para que no se sobreescriban los archivos
estacs_leidas = {}

for id_estac in estacs:
    resultado = carga_smn(id_estac)
    estacs_leidas[id_estac] = resultado

In [47]:
print(estacs_leidas['14339'].head())
print(estacs_leidas['14339'].tail())

       FECHA  PRECIP  TMAX  TMIN
0 1980-06-15     NaN   NaN   NaN
1 1980-06-16     0.0  31.0  12.0
2 1980-06-17     0.0  32.0  18.0
3 1980-06-18     0.0  34.0  18.0
4 1980-06-19     0.0  32.0  18.0
           FECHA  PRECIP  TMAX  TMIN
16180 2026-01-30     NaN   NaN   NaN
16181 2026-02-01     9.2   NaN   NaN
16182 2026-02-02     NaN   NaN   NaN
16183 2026-02-04     0.0   NaN   NaN
16184 2026-02-05     NaN   NaN   NaN


##### Control de calidad de los datos

**Límites físicos / outliers**
* Consistencia térmica: en caso de que Tmax < Tmin, se eliminarán ambos valores
* Validación de mínimo: Prcp. no puede ser negativa :. si es negativo se elimina el valor

**Estandarización de las fechas**

In [5]:
def qc_logico(df):   
    # 1. Consistencia Térmica: La Máxima no puede ser menor a la Mínima
    # Identificamos errores de captura/digitalización
    mask_temp_error = df['TMIN'] > df['TMAX']
    
    # Si TMIN > TMAX, invalidamos ambos registros asignando NaN
    df.loc[mask_temp_error, ['TMAX', 'TMIN']] = np.nan
    
    # 2. Límites Físicos: Valores negativos imposibles
    # La precipitación >= 0
    df.loc[df['PRECIP'] < 0, 'PRECIP'] = np.nan
    
    # --- INDEXACIÓN CON FECHAS ---
    # 3. Asegurar que FECHA sea tipo datetime
    df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
    
    # 4. Eliminar duplicados de fecha antes de indexar
    df = df.drop_duplicates(subset=['FECHA'])
    
    # 5. Establecer FECHA como índice
    df = df.set_index('FECHA')
    
    # 6. Ordenar cronológicamente
    df = df.sort_index()
    
    # Reporte de limpieza para seguimiento
    # Usamos len(mask_temp_error) o el ID si lo tuviéramos para identificar
    print(f"QC: {mask_temp_error.sum()} errores térmicos corregidos. Total: {len(df)} registros.")
    
    return df

# --- BUCLE PARA PROCESAR EL DICCIONARIO ---

# Creamos un nuevo diccionario para los datos limpios
estaciones_qc = {}

for id_estac, df_raw in estacs_leidas.items():
    print(f"Procesando QC para Estación {id_estac}...")
    # Aplicamos la función y guardamos el resultado
    estaciones_qc[id_estac] = qc_logico(df_raw)

# Ahora puedes acceder a cualquier estación limpia así:
# estaciones_qc['14002'].head()

Procesando QC para Estación 14002...
QC: 0 errores térmicos corregidos. Total: 22291 registros.
Procesando QC para Estación 14006...
QC: 0 errores térmicos corregidos. Total: 30209 registros.
Procesando QC para Estación 14009...
QC: 0 errores térmicos corregidos. Total: 29692 registros.
Procesando QC para Estación 14011...
QC: 0 errores térmicos corregidos. Total: 22304 registros.
Procesando QC para Estación 14016...
QC: 0 errores térmicos corregidos. Total: 26442 registros.
Procesando QC para Estación 14017...
QC: 0 errores térmicos corregidos. Total: 26760 registros.
Procesando QC para Estación 14018...
QC: 0 errores térmicos corregidos. Total: 28421 registros.
Procesando QC para Estación 14023...
QC: 0 errores térmicos corregidos. Total: 27781 registros.
Procesando QC para Estación 14024...
QC: 0 errores térmicos corregidos. Total: 23453 registros.
Procesando QC para Estación 14026...
QC: 0 errores térmicos corregidos. Total: 23671 registros.
Procesando QC para Estación 14028...
QC:

##### Integridad de la serie
**Faltantes**
* Calcular cantidad de datos faltantes
* se considerarán las estaciones que contienen más del 90% de los datos
* Calidad Mensual: Que no haya meses con más de 5 días faltantes (estos se vuelven NaN).

In [6]:
def integridad_temporal_masiva(diccionario_estaciones):
    # 1. Validar que el diccionario no esté vacío
    if not diccionario_estaciones:
        print("El diccionario de estaciones está vacío.")
        return {}

    # 2. Identificar la fecha mínima y máxima global de TODO el set de Jalisco
    # Recorremos todos los DataFrames para encontrar los extremos temporales
    fecha_min_global = min([df.index.min() for df in diccionario_estaciones.values()])
    fecha_max_global = max([df.index.max() for df in diccionario_estaciones.values()])
    
    # 3. Crear el calendario maestro diario (sin saltos)
    calendario_maestro = pd.date_range(start=fecha_min_global, end=fecha_max_global, freq='D')
    
    estaciones_normalizadas = {}
    
    # 4. Bucle para reindexar cada estación con el calendario maestro
    for id_est, df in diccionario_estaciones.items():
        # .reindex() es la clave: inserta filas con NaN en las fechas faltantes
        df_norm = df.reindex(calendario_maestro)
        df_norm.index.name = 'FECHA'
        
        # Marcamos qué días son "rellenados" (útil para auditoría de datos)
        # Si PRECIP es NaN, asumimos que es un dato faltante insertado ahora
        df_norm['MISSING_DAY'] = df_norm['PRECIP'].isna().astype(int)
        
        estaciones_normalizadas[id_est] = df_norm
        print(f"Estación {id_est}: Calendario normalizado.")
    
    print(f"\n--- NORMALIZACIÓN GLOBAL FINALIZADA ---")
    print(f"Rango de estudio Jalisco: {fecha_min_global.date()} a {fecha_max_global.date()}")
    print(f"Longitud de la serie: {len(calendario_maestro)} días.")
    
    return estaciones_normalizadas

# --- EJECUCIÓN ---

# Usamos el diccionario 'estaciones_qc' que generamos en el Bloque 2
estaciones_listas = integridad_temporal_masiva(estaciones_qc)

# Ejemplo de verificación:
#print(estaciones_listas['14002'].head())

Estación 14002: Calendario normalizado.
Estación 14006: Calendario normalizado.
Estación 14009: Calendario normalizado.
Estación 14011: Calendario normalizado.
Estación 14016: Calendario normalizado.
Estación 14017: Calendario normalizado.
Estación 14018: Calendario normalizado.
Estación 14023: Calendario normalizado.
Estación 14024: Calendario normalizado.
Estación 14026: Calendario normalizado.
Estación 14028: Calendario normalizado.
Estación 14029: Calendario normalizado.
Estación 14030: Calendario normalizado.
Estación 14032: Calendario normalizado.
Estación 14033: Calendario normalizado.
Estación 14034: Calendario normalizado.
Estación 14035: Calendario normalizado.
Estación 14036: Calendario normalizado.
Estación 14037: Calendario normalizado.
Estación 14038: Calendario normalizado.
Estación 14039: Calendario normalizado.
Estación 14040: Calendario normalizado.
Estación 14044: Calendario normalizado.
Estación 14046: Calendario normalizado.
Estación 14047: Calendario normalizado.


##### Guardar lo anterior como CSV

In [7]:
def exportar_rango_especifico(diccionario_normalizado, carpeta_salida, fecha_inicio, fecha_fin):
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    for id_est, df in diccionario_normalizado.items():
        # 1. Filtrar el DataFrame usando el índice (que es la FECHA)
        # .loc es inclusivo en ambos extremos
        try:
            df_filtrado = df.loc[fecha_inicio : fecha_fin]
            
            if df_filtrado.empty:
                print(f"Estación {id_est}: Sin datos en el rango {fecha_inicio} a {fecha_fin}. Saltando...")
                continue

            # 2. Guardar el archivo filtrado
            nombre_archivo = f"SMN_{id_est}_{fecha_inicio[:4]}_{fecha_fin[:4]}.csv"
            ruta_completa = os.path.join(carpeta_salida, nombre_archivo)
            
            df_filtrado.to_csv(ruta_completa, sep=',', encoding='utf-8', index=True)
            print(f"Estación {id_est}: Guardada con éxito ({len(df_filtrado)} días).")
            
        except Exception as e:
            print(f"Error al filtrar la estación {id_est}: {e}")

    print(f"\n--- PROCESO FINALIZADO ---")

# --- EJECUCIÓN ---
# Define tus fechas de interés (ejemplo: periodo de 30 años)
inicio = '1991-01-01'
fin = '2020-12-31'
carpeta_proyectos = "../01_DATA/SMN_limpio"

exportar_rango_especifico(estaciones_listas, carpeta_proyectos, inicio, fin)

Estación 14002: Guardada con éxito (10958 días).
Estación 14006: Guardada con éxito (10958 días).
Estación 14009: Guardada con éxito (10958 días).
Estación 14011: Guardada con éxito (10958 días).
Estación 14016: Guardada con éxito (10958 días).
Estación 14017: Guardada con éxito (10958 días).
Estación 14018: Guardada con éxito (10958 días).
Estación 14023: Guardada con éxito (10958 días).
Estación 14024: Guardada con éxito (10958 días).
Estación 14026: Guardada con éxito (10958 días).
Estación 14028: Guardada con éxito (10958 días).
Estación 14029: Guardada con éxito (10958 días).
Estación 14030: Guardada con éxito (10958 días).
Estación 14032: Guardada con éxito (10958 días).
Estación 14033: Guardada con éxito (10958 días).
Estación 14034: Guardada con éxito (10958 días).
Estación 14035: Guardada con éxito (10958 días).
Estación 14036: Guardada con éxito (10958 días).
Estación 14037: Guardada con éxito (10958 días).
Estación 14038: Guardada con éxito (10958 días).
Estación 14039: Guar

# Selección de estaciones
Las estaciones que se trabajaran son las que cumplan los siguientes criterios
a) tienen el 90% de registros (datos faltantes < 10%)
b) no pueden tener cinco registro nulos seguidos 

**Se eligió como periodo de 1991 a 2020 (30 años**)

In [9]:
# --- CONFIGURACIÓN DE RUTAS ---
PATH_ENTRADA = "../01_DATA/SMN_limpio/"
PATH_SALIDA = "../01_DATA/for_indices"
FECHA_INICIO = "1991-01-01"
FECHA_FIN = "2020-12-31"

def tiene_rachas_nan(serie, max_consecutivos=10):
    # Creamos una serie booleana (True donde es NaN)
    es_nan = serie.isna()
    # Identificamos bloques consecutivos de True
    # (Comparamos el valor con el anterior para marcar cambios de bloque)
    bloques = es_nan.ne(es_nan.shift()).cumsum()
    # Contamos el tamaño de cada bloque de NaNs
    tamaño_bloques = es_nan.groupby(bloques).transform('sum')
    # Si algún bloque de NaNs es >= al límite, devolvemos True
    return (tamaño_bloques >= max_consecutivos).any()

def filtrar_estaciones_estricto():
    archivos = [f for f in os.listdir(PATH_ENTRADA) if f.endswith('.csv')]
    aptas = 0
    
    print(f"Iniciando filtrado estricto de {len(archivos)} estaciones...")

    for archivo in archivos:
        df = pd.read_csv(os.path.join(PATH_ENTRADA, archivo), 
                         index_col='FECHA', parse_dates=True)
        
        # 1. Recorte temporal
        df_p = df.loc[FECHA_INICIO : FECHA_FIN]
        
        # 2. Validación de existencia de datos
        if df_p.empty or len(df_p) < 1000: # Filtro básico de seguridad
            continue

        # --- CRITERIOS DE CALIDAD ---
        
        # A. Máximo 10% de faltantes totales
        pct_nan = (df_p['PRECIP'].isna().sum() / len(df_p)) * 100
        
        # B. No más de 5 días SEGUIDOS con NaN
        racha_nan = tiene_rachas_nan(df_p['PRECIP'], max_consecutivos=10)
        
        # C. No más de 5 días faltantes en UN MES (alternos o no)
        # Esto asegura que cada mes sea representativo para el SPI
        meses_invalidos = (df_p['PRECIP'].isna().resample('MS').sum() > 10).any()

        # --- DECISIÓN FINAL ---
        if pct_nan <= 20 and not racha_nan and not meses_invalidos:
            ruta_destino = os.path.join(PATH_SALIDA, archivo)
            df_p.to_csv(ruta_destino)
            aptas += 1
            print(f"✅ {archivo}: APTO (Faltantes: {pct_nan:.1f}%)")
        else:
            razon = []
            if pct_nan > 20: razon.append(f"Total NaNs > 20% ({pct_nan:.1f}%)")
            if racha_nan: razon.append("Racha >= 10 días NaN")
            if meses_invalidos: razon.append("Meses con 10 NaNs")
            print(f"❌ {archivo}: Descartado por {', '.join(razon)}")

    print(f"\n--- FILTRADO TERMINADO ---")
    print(f"Estaciones finales en 'for_indices': {aptas}")

filtrar_estaciones_estricto()

Iniciando filtrado estricto de 125 estaciones...
✅ SMN_14002_1991_2020.csv: APTO (Faltantes: 0.0%)
❌ SMN_14006_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14009_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14011_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14016_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14017_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14018_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14023_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14024_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14026_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14028_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs
❌ SMN_14029_1991_2020.csv: Descartado por Racha >= 10 días NaN, Meses con 10 NaNs